In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,2020-11-18T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1528,240,1768,30334,...,1346,1837,13411,1015,28823.0,17705.0,46528,845013,582685.0,NaN
5645,2020-11-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,1805,282,2087,52023,...,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0,effettuati 50901 tamponi rapidi antigenici di...
5646,2020-11-18T17:00:00,ITA,10,Umbria,43.106758,12.388247,368,74,442,10832,...,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0,NaN
5647,2020-11-18T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,146,17,163,1930,...,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,2020-11-18T17:00:00,19,Sicilia,1528,240,1768,30334,32102,1346,1837,13411,1015,28823.0,17705.0,46528,845013,582685.0
5645,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0
5646,2020-11-18T17:00:00,10,Umbria,368,74,442,10832,11274,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0
5647,2020-11-18T17:00:00,2,Valle d'Aosta,146,17,163,1930,2093,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-18', '2020-11-17')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-18T17:00:00,1,Piemonte,5208,388,5596,69468,75064,1278,3281,54511,5253,56287.0,78541.0,134828,1322809,836881.0
1,2020-11-18T17:00:00,2,Valle d'Aosta,146,17,163,1930,2093,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0
2,2020-11-18T17:00:00,3,Lombardia,8323,903,9226,142857,152083,-4484,7633,169056,19850,257237.0,83752.0,340989,3644914,2193319.0
3,2020-11-18T17:00:00,5,Veneto,2079,274,2353,61953,64306,1235,2972,41613,3019,28335.0,80603.0,108938,2607348,1005997.0
4,2020-11-18T17:00:00,6,Friuli Venezia Giulia,470,49,519,10235,10754,333,796,9404,567,17665.0,3060.0,20725,636812,257516.0
5,2020-11-18T17:00:00,7,Liguria,1371,118,1489,15279,16768,237,775,26271,2163,31254.0,13948.0,45202,542028,271234.0
6,2020-11-18T17:00:00,8,Emilia-Romagna,2449,244,2693,56626,59319,2051,2371,31167,5168,61355.0,34299.0,95654,1916548,995472.0
7,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0
8,2020-11-18T17:00:00,10,Umbria,368,74,442,10832,11274,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0
9,2020-11-18T17:00:00,11,Marche,527,79,606,14147,14753,242,479,8542,1139,18594.0,5840.0,24434,377624,220387.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-17T17:00:00,1,Piemonte,5150,384,5534,68252,73786,846,2606,52571,5190,55023.0,76524.0,131547,1305054,826436.0
1,2020-11-17T17:00:00,2,Valle d'Aosta,152,17,169,1994,2163,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0
2,2020-11-17T17:00:00,3,Lombardia,8151,894,9045,147522,156567,4228,8448,157121,19668,251767.0,81589.0,333356,3606814,2177539.0
3,2020-11-17T17:00:00,5,Veneto,1980,266,2246,60825,63071,373,3124,39928,2967,27936.0,78030.0,105966,2590427,1000577.0
4,2020-11-17T17:00:00,6,Friuli Venezia Giulia,454,44,498,9923,10421,366,536,8952,556,17057.0,2872.0,19929,629511,255770.0
5,2020-11-17T17:00:00,7,Liguria,1393,117,1510,15021,16531,630,685,25752,2144,30766.0,13661.0,44427,535222,268832.0
6,2020-11-17T17:00:00,8,Emilia-Romagna,2451,247,2698,54570,57268,1839,2219,30902,5115,60144.0,33141.0,93285,1895816,988202.0
7,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0
8,2020-11-17T17:00:00,10,Umbria,368,70,438,10806,11244,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0
9,2020-11-17T17:00:00,11,Marche,525,78,603,13908,14511,300,357,8314,1130,23955.0,0.0,23955,372822,218598.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-18T17:00:00,1,Piemonte,5208,388,5596,69468,75064,1278,3281,54511,5253,56287.0,78541.0,134828,1322809,836881.0
1,2020-11-18T17:00:00,2,Valle d'Aosta,146,17,163,1930,2093,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0
2,2020-11-18T17:00:00,3,Lombardia,8323,903,9226,142857,152083,-4484,7633,169056,19850,257237.0,83752.0,340989,3644914,2193319.0
3,2020-11-18T17:00:00,4,Trentino-Alto Adige,879,77,956,12805,13761,66,817,17211,974,25667.0,6279.0,31946,620271,NaN
4,2020-11-18T17:00:00,5,Veneto,2079,274,2353,61953,64306,1235,2972,41613,3019,28335.0,80603.0,108938,2607348,1005997.0
5,2020-11-18T17:00:00,6,Friuli Venezia Giulia,470,49,519,10235,10754,333,796,9404,567,17665.0,3060.0,20725,636812,257516.0
6,2020-11-18T17:00:00,7,Liguria,1371,118,1489,15279,16768,237,775,26271,2163,31254.0,13948.0,45202,542028,271234.0
7,2020-11-18T17:00:00,8,Emilia-Romagna,2449,244,2693,56626,59319,2051,2371,31167,5168,61355.0,34299.0,95654,1916548,995472.0
8,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0
9,2020-11-18T17:00:00,10,Umbria,368,74,442,10832,11274,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-17T17:00:00,1,Piemonte,5150,384,5534,68252,73786,846,2606,52571,5190,55023.0,76524.0,131547,1305054,826436.0
1,1,2020-11-17T17:00:00,2,Valle d'Aosta,152,17,169,1994,2163,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0
2,2,2020-11-17T17:00:00,3,Lombardia,8151,894,9045,147522,156567,4228,8448,157121,19668,251767.0,81589.0,333356,3606814,2177539.0
3,19,2020-11-17T17:00:00,4,Trentino-Alto Adige,870,79,949,12746,13695,289,534,16478,956,24981.0,6148.0,31129,612579,NaN
4,3,2020-11-17T17:00:00,5,Veneto,1980,266,2246,60825,63071,373,3124,39928,2967,27936.0,78030.0,105966,2590427,1000577.0
5,4,2020-11-17T17:00:00,6,Friuli Venezia Giulia,454,44,498,9923,10421,366,536,8952,556,17057.0,2872.0,19929,629511,255770.0
6,5,2020-11-17T17:00:00,7,Liguria,1393,117,1510,15021,16531,630,685,25752,2144,30766.0,13661.0,44427,535222,268832.0
7,6,2020-11-17T17:00:00,8,Emilia-Romagna,2451,247,2698,54570,57268,1839,2219,30902,5115,60144.0,33141.0,93285,1895816,988202.0
8,7,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0
9,8,2020-11-17T17:00:00,10,Umbria,368,70,438,10806,11244,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-18T17:00:00,1,Piemonte,5208,388,5596,69468,75064,1278,3281,54511,5253,56287.0,78541.0,134828,1322809,836881.0,4341375
1,2020-11-18T17:00:00,2,Valle d'Aosta,146,17,163,1930,2093,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0,125501
2,2020-11-18T17:00:00,3,Lombardia,8323,903,9226,142857,152083,-4484,7633,169056,19850,257237.0,83752.0,340989,3644914,2193319.0,10103969
3,2020-11-18T17:00:00,4,Trentino-Alto Adige,879,77,956,12805,13761,66,817,17211,974,25667.0,6279.0,31946,620271,NaN,1074819
4,2020-11-18T17:00:00,5,Veneto,2079,274,2353,61953,64306,1235,2972,41613,3019,28335.0,80603.0,108938,2607348,1005997.0,4907704
5,2020-11-18T17:00:00,6,Friuli Venezia Giulia,470,49,519,10235,10754,333,796,9404,567,17665.0,3060.0,20725,636812,257516.0,1211357
6,2020-11-18T17:00:00,7,Liguria,1371,118,1489,15279,16768,237,775,26271,2163,31254.0,13948.0,45202,542028,271234.0,1543127
7,2020-11-18T17:00:00,8,Emilia-Romagna,2449,244,2693,56626,59319,2051,2371,31167,5168,61355.0,34299.0,95654,1916548,995472.0,4467118
8,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0,3722729
9,2020-11-18T17:00:00,10,Umbria,368,74,442,10832,11274,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-18T17:00:00,1,Piemonte,5208,388,5596,69468,75064,1278,3281,54511,5253,56287.0,78541.0,134828,1322809,836881.0,4341375,575
1,2020-11-18T17:00:00,2,Valle d'Aosta,146,17,163,1930,2093,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0,125501,20
2,2020-11-18T17:00:00,3,Lombardia,8323,903,9226,142857,152083,-4484,7633,169056,19850,257237.0,83752.0,340989,3644914,2193319.0,10103969,1036
3,2020-11-18T17:00:00,4,Trentino-Alto Adige,879,77,956,12805,13761,66,817,17211,974,25667.0,6279.0,31946,620271,NaN,1074819,106
4,2020-11-18T17:00:00,5,Veneto,2079,274,2353,61953,64306,1235,2972,41613,3019,28335.0,80603.0,108938,2607348,1005997.0,4907704,1016
5,2020-11-18T17:00:00,6,Friuli Venezia Giulia,470,49,519,10235,10754,333,796,9404,567,17665.0,3060.0,20725,636812,257516.0,1211357,180
6,2020-11-18T17:00:00,7,Liguria,1371,118,1489,15279,16768,237,775,26271,2163,31254.0,13948.0,45202,542028,271234.0,1543127,209
7,2020-11-18T17:00:00,8,Emilia-Romagna,2449,244,2693,56626,59319,2051,2371,31167,5168,61355.0,34299.0,95654,1916548,995472.0,4467118,634
8,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0,3722729,523
9,2020-11-18T17:00:00,10,Umbria,368,74,442,10832,11274,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-18T17:00:00,3,Lombardia,8323,903,9226,142857,152083,-4484,7633,...,87.16,-4665,-815,38100,15780.0,182,11935,20.03,0.07554,3.37
1,2020-11-18T17:00:00,15,Campania,2259,200,2459,92680,95139,2413,3657,...,39.60,2391,638,23479,19634.0,75,1169,15.58,0.06321,2.11
2,2020-11-18T17:00:00,1,Piemonte,5208,388,5596,69468,75064,1278,3281,...,67.48,1216,675,17755,10445.0,63,1940,18.48,0.07558,3.11
3,2020-11-18T17:00:00,5,Veneto,2079,274,2353,61953,64306,1235,2972,...,26.97,1128,-152,16921,5420.0,52,1685,17.56,0.06056,2.22
4,2020-11-18T17:00:00,12,Lazio,3056,318,3374,70117,73491,2236,2866,...,37.54,2236,328,26869,21558.0,80,550,10.67,0.04886,1.57
5,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,...,53.92,-60,147,22365,10475.0,55,2495,11.21,0.06737,2.33
6,2020-11-18T17:00:00,8,Emilia-Romagna,2449,244,2693,56626,59319,2051,2371,...,38.49,2056,152,20732,7270.0,53,265,11.44,0.05308,2.14
7,2020-11-18T17:00:00,19,Sicilia,1528,240,1768,30334,32102,1346,1837,...,40.82,1310,139,9479,4867.0,44,447,19.38,0.03697,0.94
8,2020-11-18T17:00:00,16,Puglia,1320,198,1518,26096,27614,1007,1368,...,53.66,986,134,10265,6047.0,28,333,13.33,0.03413,0.95
9,2020-11-18T17:00:00,18,Calabria,388,46,434,8097,8531,793,936,...,19.25,765,256,5906,4662.0,10,133,15.85,0.04863,0.62


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,903,133,1036,87.16
1,Valle d'Aosta,17,3,20,85.00
2,Trentino-Alto Adige,77,29,106,72.64
3,Piemonte,388,187,575,67.48
4,Umbria,74,43,117,63.25
5,Liguria,118,91,209,56.46
6,Marche,79,64,143,55.24
7,Toscana,282,241,523,53.92
8,Puglia,198,171,369,53.66
9,Abruzzo,65,83,148,43.92


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
